Please note that this notebook is the basis for an importable class stored in `kediff_ner_system.py`. Subsequent notebooks or classes **will use that script**, while the present notebook only served for constructing that class. ⚠️

# Load'n'Import

## Packages

In [1]:
import os

import tabulate
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer, pipeline

## Define `DATA_DIR`

In [2]:
try:
    from google.colab import drive

    print(
        "You work on Colab. Gentle as we are, we will mount Drive for you. "
        "It'd help if you allowed this in the popup that opens."
    )
    drive.mount('/content/drive')
    DATA_DIR = os.path.join('drive', 'MyDrive', 'KEDiff', 'data')
except ModuleNotFoundError:
    print("You do not work on Colab")
    DATA_DIR = os.path.join('data')
print(f"{DATA_DIR=}", '-->', os.path.abspath(DATA_DIR))

You work on Colab. Gentle as we are, we will mount Drive for you. It'd help if you allowed this in the popup that opens.
Mounted at /content/drive
DATA_DIR='drive/MyDrive/KEDiff/data' --> /content/drive/MyDrive/KEDiff/data


## Load _hmBERT_ Tokeniser `dbmdz/bert-base-historic-multilingual-cased`

In [3]:
TOKENISER_CHECKPOINT = "dbmdz/bert-base-historic-multilingual-cased"
CHECKPOINT_NAME_BASE = "oalz-1788-q1-ner-"
TRAINED_DIR = os.path.join(DATA_DIR, 'trained_models', '2024-01-15')

In [4]:
tokeniser = AutoTokenizer.from_pretrained(TOKENISER_CHECKPOINT)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/561 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/212k [00:00<?, ?B/s]

Test the tokeniser.

In [5]:
sample = {
    "id": 1003,
    "text": "Unter die Gegenden, die den Borkenk\u00e4fer n\u00e4hren S. 13 \u2014 14. geh\u00f6rt leider auch das Erzstift Salzburg.",
    "label": [[82, 99, "ORG"], [82, 99, "LOC"], [28, 39, "MISC"]]
}
sample

{'id': 1003,
 'text': 'Unter die Gegenden, die den Borkenkäfer nähren S. 13 — 14. gehört leider auch das Erzstift Salzburg.',
 'label': [[82, 99, 'ORG'], [82, 99, 'LOC'], [28, 39, 'MISC']]}

In [6]:
x = tokeniser(sample['text'])
x

{'input_ids': [2, 2558, 788, 25854, 668, 16, 788, 767, 3383, 1199, 1176, 1582, 3835, 2221, 55, 18, 1097, 338, 1136, 18, 16130, 24914, 1532, 1190, 7840, 14369, 18843, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

## Define some sample texts to run NER on

In [7]:
sample_texts = [
    "(Das hei\u00dft ab ovo anfangen, wie's jener that, der vom deutschen Gleichgewichte handeln wollte, und von Adam anfieng.)",
    "Daniel Göller ist der beste Masterstudent Christian Borgelts und sollte eine Ehrenmedaille… medallje… medallie… wie schreibt man das????… sowie eine saftige Sonderzahlung von Herrn Prof. Lehnert, Rektor der Universität zu Salzburg, erhalten, sodass er endlich nach Island reisen und dort in einer Kirche zu Gott beten kann.",
    "Nun ist die Frage, ob das Modell auch mit Frauennamen umgehen kann, da beim Lesen der Originaltexte ein deutlicher Bias zu Männernamen aufgefallen ist. Und wie es dann wohl mit geschlechtsneutralen Namen aussieht?",
    "Bundeskanzlerin Brigitte Bierlein führte bis zur Angelobung der Bundesregierung Kurz II nach der vorgezogenen Nationalratswahl im Herbst 2019 die Amtsgeschäfte der Bundesministerien weiter. Vielleicht kennt sie ja auch Angela Merkel?",
    "Test in Salzburg während der Österreichsichen Aufklärung. In Paris wurden mehrere Menschen aus Deutschland gesichtet.",
    "den meisten Lesern durch eine ausführliche Beschreibung und Beurtheilung des Wirtembergischen, im katholischen Deutschlande noch immer nicht genug belannten Gesangbuches, einen Gefallen zu erzeigen."
]

## Load NER systems

In [8]:
label_types = ["EVENT", "LOC", "MISC", "ORG", "PER", "TIME"]
selected_epochs = {
    "EVENT" : "checkpoint-1393",
    "LOC" : "checkpoint-1393",
    "MISC" : "checkpoint-2786",
    "ORG" : "checkpoint-1393",
    "PER" : "checkpoint-2786",
    "TIME" : "checkpoint-1393"
}
ner_model_paths = {
    label_type: os.path.join(TRAINED_DIR,
                             "".join([CHECKPOINT_NAME_BASE, label_type]),
                             selected_epochs[label_type])
    for label_type in label_types
}
ner_model_paths

{'EVENT': 'drive/MyDrive/KEDiff/data/trained_models/2024-01-15/oalz-1788-q1-ner-EVENT/checkpoint-1393',
 'LOC': 'drive/MyDrive/KEDiff/data/trained_models/2024-01-15/oalz-1788-q1-ner-LOC/checkpoint-1393',
 'MISC': 'drive/MyDrive/KEDiff/data/trained_models/2024-01-15/oalz-1788-q1-ner-MISC/checkpoint-2786',
 'ORG': 'drive/MyDrive/KEDiff/data/trained_models/2024-01-15/oalz-1788-q1-ner-ORG/checkpoint-1393',
 'PER': 'drive/MyDrive/KEDiff/data/trained_models/2024-01-15/oalz-1788-q1-ner-PER/checkpoint-2786',
 'TIME': 'drive/MyDrive/KEDiff/data/trained_models/2024-01-15/oalz-1788-q1-ner-TIME/checkpoint-1393'}

In [9]:
token_classifiers = {}
for label_type in tqdm(label_types):
    token_classifiers[label_type] = pipeline(
        "token-classification",
        model=os.path.abspath(ner_model_paths[label_type]),
        aggregation_strategy="simple"
    )
list(token_classifiers.keys())

100%|██████████| 6/6 [00:57<00:00,  9.50s/it]


['EVENT', 'LOC', 'MISC', 'ORG', 'PER', 'TIME']

# Build NER pipeline

## Sub functions

In [10]:
def find_entities(text: str):
    entities = {
        label_type : token_classifiers[label_type](text)
        for label_type in label_types
    }
    return entities

In [11]:
text = sample_texts[0]
entities_dict = find_entities(text)
entities_dict

{'EVENT': [],
 'LOC': [{'entity_group': 'LOC',
   'score': 0.89277846,
   'word': 'deutschen',
   'start': 54,
   'end': 63}],
 'MISC': [],
 'ORG': [],
 'PER': [{'entity_group': 'PER',
   'score': 0.98266566,
   'word': 'Adam',
   'start': 103,
   'end': 107}],
 'TIME': []}

In [12]:
def entities_dict_to_list(entities_dict):
    entities_list = []
    for label_type in entities_dict:
        entities_list += entities_dict[label_type]
    return entities_list

In [13]:
entity_list = entities_dict_to_list(entities_dict)
entity_list

[{'entity_group': 'LOC',
  'score': 0.89277846,
  'word': 'deutschen',
  'start': 54,
  'end': 63},
 {'entity_group': 'PER',
  'score': 0.98266566,
  'word': 'Adam',
  'start': 103,
  'end': 107}]

In [14]:
def sort_entity_list(entity_list):
    sorted_entity_list = sorted(entity_list, key=lambda d: (d['start'], d['score'], d['entity_group']))
    return sorted_entity_list

In [15]:
sorted_entity_list = sort_entity_list(entity_list)
sorted_entity_list

[{'entity_group': 'LOC',
  'score': 0.89277846,
  'word': 'deutschen',
  'start': 54,
  'end': 63},
 {'entity_group': 'PER',
  'score': 0.98266566,
  'word': 'Adam',
  'start': 103,
  'end': 107}]

In [16]:
def print_entities_as_table(entity_list, tablefmt: str = "simple_outline"):
    if entity_list is None or len(entity_list) == 0:
        print("No entities found")
        return
    header = list(entity_list[0].keys())
    header[0] = "type"
    rows = [entity.values() for entity in entity_list]
    print(tabulate.tabulate(rows, header, tablefmt=tablefmt))

In [17]:
print_entities_as_table(sorted_entity_list)

┌────────┬──────────┬───────────┬─────────┬───────┐
│ type   │    score │ word      │   start │   end │
├────────┼──────────┼───────────┼─────────┼───────┤
│ LOC    │ 0.892778 │ deutschen │      54 │    63 │
│ PER    │ 0.982666 │ Adam      │     103 │   107 │
└────────┴──────────┴───────────┴─────────┴───────┘


## Stick them together into a pipeline

In [18]:
def ner(text: str, print_table: bool = False):
    entity_dict = find_entities(text)
    entity_list = entities_dict_to_list(entity_dict)
    sorted_entity_list = sort_entity_list(entity_list)
    if print_table: print_entities_as_table(sorted_entity_list)
    return sorted_entity_list

In [19]:
entities = [ner(text) for text in tqdm(sample_texts)]
entities

100%|██████████| 6/6 [00:09<00:00,  1.54s/it]


[[{'entity_group': 'LOC',
   'score': 0.89277846,
   'word': 'deutschen',
   'start': 54,
   'end': 63},
  {'entity_group': 'PER',
   'score': 0.98266566,
   'word': 'Adam',
   'start': 103,
   'end': 107}],
 [{'entity_group': 'PER',
   'score': 0.9947171,
   'word': 'Daniel Göller',
   'start': 0,
   'end': 13},
  {'entity_group': 'PER',
   'score': 0.975671,
   'word': 'Christian Borgelts',
   'start': 42,
   'end': 60},
  {'entity_group': 'PER',
   'score': 0.84162706,
   'word': 'Herrn Prof. Lehnert, Rektor der Universität',
   'start': 175,
   'end': 218},
  {'entity_group': 'ORG',
   'score': 0.8358596,
   'word': 'Universität zu Salzburg',
   'start': 207,
   'end': 230},
  {'entity_group': 'LOC',
   'score': 0.9155426,
   'word': 'Salzburg',
   'start': 222,
   'end': 230},
  {'entity_group': 'LOC',
   'score': 0.9196716,
   'word': 'Island',
   'start': 265,
   'end': 271},
  {'entity_group': 'ORG',
   'score': 0.61161256,
   'word': 'Kirche',
   'start': 297,
   'end': 303},


In [20]:
i = 1
print(sample_texts[i])
print("")
x = ner(sample_texts[i], print_table=True)

Daniel Göller ist der beste Masterstudent Christian Borgelts und sollte eine Ehrenmedaille… medallje… medallie… wie schreibt man das????… sowie eine saftige Sonderzahlung von Herrn Prof. Lehnert, Rektor der Universität zu Salzburg, erhalten, sodass er endlich nach Island reisen und dort in einer Kirche zu Gott beten kann.

┌────────┬──────────┬─────────────────────────────────────────────┬─────────┬───────┐
│ type   │    score │ word                                        │   start │   end │
├────────┼──────────┼─────────────────────────────────────────────┼─────────┼───────┤
│ PER    │ 0.994717 │ Daniel Göller                               │       0 │    13 │
│ PER    │ 0.975671 │ Christian Borgelts                          │      42 │    60 │
│ PER    │ 0.841627 │ Herrn Prof. Lehnert, Rektor der Universität │     175 │   218 │
│ ORG    │ 0.83586  │ Universität zu Salzburg                     │     207 │   230 │
│ LOC    │ 0.915543 │ Salzburg                                    │     2

In [21]:
x = ner("Willkommen in Österreich", print_table=True)

┌────────┬──────────┬────────────┬─────────┬───────┐
│ type   │    score │ word       │   start │   end │
├────────┼──────────┼────────────┼─────────┼───────┤
│ LOC    │ 0.961031 │ Österreich │      14 │    24 │
└────────┴──────────┴────────────┴─────────┴───────┘


# Some observations when using the NER system

In [22]:
x = ner("D. Hr. Rec. betrachtet den Text von L.K.Y.I. aus der vorhergehenden Ausgabe.", print_table=True)

┌────────┬──────────┬────────────┬─────────┬───────┐
│ type   │    score │ word       │   start │   end │
├────────┼──────────┼────────────┼─────────┼───────┤
│ PER    │ 0.991839 │ Hr. Rec.   │       3 │    11 │
│ MISC   │ 0.571993 │ L. K. Y    │      36 │    41 │
│ PER    │ 0.979722 │ L. K. Y. I │      36 │    43 │
└────────┴──────────┴────────────┴─────────┴───────┘


In [23]:
x = ner("D. Hr. V. betrachtet den Text von L.K.Y.I. aus der vorhergehenden Ausgabe.", print_table=True)

┌────────┬──────────┬────────────┬─────────┬───────┐
│ type   │    score │ word       │   start │   end │
├────────┼──────────┼────────────┼─────────┼───────┤
│ PER    │ 0.969234 │ Hr. V.     │       3 │     9 │
│ PER    │ 0.983884 │ L. K. Y. I │      34 │    41 │
└────────┴──────────┴────────────┴─────────┴───────┘


In [24]:
x = ner("Der V. betrachtet den Text von L.K.Y.I. aus der vorhergehenden Ausgabe.", print_table=True)

┌────────┬──────────┬────────────┬─────────┬───────┐
│ type   │    score │ word       │   start │   end │
├────────┼──────────┼────────────┼─────────┼───────┤
│ PER    │ 0.996371 │ V          │       4 │     5 │
│ PER    │ 0.969254 │ L. K. Y. I │      31 │    38 │
│ MISC   │ 0.523102 │ K          │      33 │    34 │
└────────┴──────────┴────────────┴─────────┴───────┘


Als Rec. ist LKYI MISC mit Verfasser nicht. 🤔

In [25]:
x = ner("D. Hr. Verf. betrachtet den Text von L.K.Y.I. aus der vorhergehenden Ausgabe.", print_table=True)

┌────────┬──────────┬────────────┬─────────┬───────┐
│ type   │    score │ word       │   start │   end │
├────────┼──────────┼────────────┼─────────┼───────┤
│ PER    │ 0.993812 │ Hr. Verf.  │       3 │    12 │
│ MISC   │ 0.554557 │ L. K. Y.   │      37 │    43 │
│ PER    │ 0.978283 │ L. K. Y. I │      37 │    44 │
└────────┴──────────┴────────────┴─────────┴───────┘


In [26]:
tokeniser("Kantsche Theorie")

{'input_ids': [2, 24626, 3040, 922, 12942, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [27]:
tokeniser("Kantsche")

{'input_ids': [2, 24626, 3040, 3], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

# Thoughts & Todos

Confusion matrix:

* we've got 3 classes: `O`, `B-PER`, `I-PER`
* transform to `O` and `PER` → treat as **binary classification** matrix by summarising `B-PER` and `I-PER` in same class "`PER`".

**How do we count? Is our unit a word, tokens, something else?**

```
Kantsche Theorie
PPPP------------

Daniels Dominik
PPPPPPP-PPPPPPP
BIIIIII-BIIIIII
```

Todos

*   create an index of persons and the pages they're mentioned on
*   adds an outlook for other tasks on the manual labour to be done after automatic index creation (PER is class with most data)
*   transform automatic annotations (step4) to LATEX format (OCR files) and XML
*   make models available, e.g. on the HF Model Hub --> plus example to play around with
*   write thesis